In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report

In [3]:
def separate_dataset(dataframe, target):
    x = dataframe.drop(columns=[target]).values
    y = dataframe[target].values
    return x, y

In [4]:
def calc_classes_prob(dataframe, target):
    prob = {}
    total_count = len(dataframe)
    for target_class in dataframe[target].unique():
        prob[target_class] = len(dataframe[dataframe[
                                               target] == target_class]) / total_count  # comment division to use count instead of probability
    return prob

In [5]:
def calc_feature_in_class_prob(dataframe, feature, feature_value, target, target_class):
    count_of_feature_in_class_with_value = len(
        dataframe[(dataframe[target] == target_class) & (dataframe[feature] == feature_value)])
    count_of_feature_in_class = len(dataframe[dataframe[target] == target_class])
    return count_of_feature_in_class_with_value / count_of_feature_in_class  # comment division to use count instead of probability

In [6]:
def predict(dataframe, examples, target):
    classes = dataframe[target].unique()
    prob_for_classes = calc_classes_prob(dataframe, target)
    predictions = []
    for ex in examples:
        prob = {}
        for cls in classes:
            cls_prob = prob_for_classes[cls]
            for feature, value in ex.items():
                if feature != target:
                    feature_in_class_prob = calc_feature_in_class_prob(dataframe, feature, value, target, cls)
                    cls_prob *= feature_in_class_prob
            prob[cls] = cls_prob
        predicted_class = max(prob, key=prob.get)
        predictions.append(predicted_class)
    return predictions

In [7]:
def create_test_examples(dataframe, target):
    feature_columns = [col for col in dataframe.columns if col != target]
    return [dict(zip(feature_columns, example)) for example in dataframe[feature_columns].values]

In [8]:
# cols = [
#     'class', 'cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor',
#     'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color',
#     'stalk-shape', 'stalk-root', 'stalk-surface-above-ring',
#     'stalk-surface-below-ring', 'stalk-color-above-ring',
#     'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number',
#     'ring-type', 'spore-print-color', 'population', 'habitat'
# ]
#
# df = pd.read_csv('datasets/agaricus-lepiota.data', names=cols)
# target_feature = 'class'
#
cols = [
    'class', 'handicapped-infants', 'water-project-cost-sharing',
    'adoption-of-the-budget-resolution', 'physician-fee-freeze',
    'el-salvador-aid', 'religious-groups-in-schools', 'anti-satellite-test-ban',
    'aid-to-nicaraguan-contras', 'mx-missile', 'immigration',
    'synfuels-corporation-cutback', 'education-spending', 'superfund-right-to-sue',
    'crime', 'duty-free-exports', 'export-administration-act-south-africa'
]

df = pd.read_csv('datasets/house-votes-84.data', names=cols)
target_feature = 'class'
#
# cols = [
#     'a1', 'a2', 'a3', 'a4', 'a5', 'a6',
#     'b1', 'b2', 'b3', 'b4', 'b5', 'b6',
#     'c1', 'c2', 'c3', 'c4', 'c5', 'c6',
#     'd1', 'd2', 'd3', 'd4', 'd5', 'd6',
#     'e1', 'e2', 'e3', 'e4', 'e5', 'e6',
#     'f1', 'f2', 'f3', 'f4', 'f5', 'f6',
#     'g1', 'g2', 'g3', 'g4', 'g5', 'g6',
#     'class'
# ]
#
# df = pd.read_csv('datasets/connect-4.data', names=cols)
# target_feature = 'class'
#
# cols = [
#     'parents', 'has_nurs', 'form', 'children', 'housing', 'finance', 'social', 'health', 'class'
# ]
#
# df = pd.read_csv('datasets/nursery.data', names=cols)
# target_feature = 'class'
#
# cols = [
#     'bkblk', 'bknwy', 'bkon8', 'bkona', 'bkspr', 'bkxbq', 'bkxcr', 'bkxwp', 'blxwp', 'bxqsq',
#     'cntxt', 'dsopp', 'dwipd', 'hdchk', 'katri', 'mulch', 'qxmsq', 'r2ar8', 'reskd', 'reskr',
#     'rimmx', 'rkxwp', 'rxmsq', 'simpl', 'skach', 'skewr', 'skrxp', 'spcop', 'stlmt', 'thrsk',
#     'wkcti', 'wkna8', 'wknck', 'wkovl', 'wkpos', 'wtoeg'
# ]
#
# df = pd.read_csv('datasets/kr-vs-kp.data', names=cols)
# target_feature = 'wtoeg'
#
# cols = ['class', 'age', 'menopause', 'tumor-size', 'inv-nodes', 'node-caps',
#         'deg-malig', 'breast', 'breast-quad', 'irradiat']

# df = pd.read_csv('datasets/breast-cancer.data', names=cols)
# target_feature = 'class'
#
# cols = [
#     'class',
#     'date', 'plant-stand', 'precip', 'temp', 'hail', 'crop-hist', 'area-damaged',
#     'severity', 'seed-tmt', 'germination', 'plant-growth', 'leaves', 'leafspots-halo',
#     'leafspots-marg', 'leafspot-size', 'leaf-shread', 'leaf-malf', 'leaf-mild', 'stem',
#     'lodging', 'stem-cankers', 'canker-lesion', 'fruiting-bodies', 'external-decay',
#     'mycelium', 'int-discolor', 'sclerotia', 'fruit-pods', 'fruit-spots', 'seed',
#     'mold-growth', 'seed-discolor', 'seed-size', 'shriveling', 'roots'
# ]
#
# df = pd.read_csv('datasets/soybean-large.data', names=cols)
# target_feature = 'class'

In [9]:
train, test = np.split(df.sample(frac=1, random_state=42), [int(0.7 * len(df))])

c:\Users\Oleksa\desktop\IAD-labs\.venv\Lib\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [10]:
train = pd.DataFrame(train, columns=cols)
test = pd.DataFrame(test, columns=cols)

In [11]:
X_test, y_test = separate_dataset(test, target_feature)
test_examples = create_test_examples(test, target_feature)

In [12]:
y_predictions = predict(train, test_examples, target_feature)
print(classification_report(y_test, y_predictions))

              precision    recall  f1-score   support

    democrat       0.96      0.87      0.91        79
  republican       0.83      0.94      0.88        52

    accuracy                           0.90       131
   macro avg       0.89      0.91      0.90       131
weighted avg       0.91      0.90      0.90       131

